In [1]:
import warnings
import pprint
import skrebate
import imblearn
from imblearn import under_sampling, over_sampling, combine
from imblearn.pipeline import Pipeline as imbPipeline
from sklearn import (preprocessing, svm, linear_model, ensemble, naive_bayes,
                    tree, neighbors, decomposition, kernel_approximation, cluster)
from sklearn.pipeline import Pipeline
from sklearn.base import clone

from sklearn.compose import TransformedTargetRegressor
from sklearn.model_selection import (KFold, GroupKFold, StratifiedKFold,
                                    LeaveOneGroupOut, cross_validate,
                                    cross_val_predict, learning_curve,
                                    GridSearchCV)
from sklearn.feature_selection import SelectKBest, f_regression, SelectFromModel, VarianceThreshold, f_classif
from sklearn.metrics import (r2_score, auc, roc_auc_score, balanced_accuracy_score, 
                             average_precision_score, confusion_matrix, roc_curve,
                             precision_recall_curve)
from sklearn.metrics.scorer import roc_auc_scorer
from sklearn.preprocessing import QuantileTransformer, quantile_transform, StandardScaler, MinMaxScaler
from sklearn.utils.class_weight import compute_class_weight, compute_sample_weight
from sklearn.utils.validation import check_memory
from xgboost import XGBRegressor, XGBClassifier
from sklearn.ensemble import RandomForestClassifier

warnings.simplefilter('ignore')

/Users/guq/galaxy/.venv/lib/python2.7/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/Users/guq/galaxy/.venv/lib/python2.7/site-packages/matplotlib/mpl-data/matplotlibrc", line #620
  (fname, cnt))


In [2]:
import os
import sys

In [3]:
import numpy as np
import pandas as pd
import re


import plotly.plotly as py
import plotly.graph_objs as go
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

## result

In [4]:
work_dir = './drug_respond/results/smmart_proten_rna_tissue10/'
sub1 = 'Hyperparameter Search on collection 18 _ randomforest/'
sub2 = 'Hyperparameter Search on collection 18 _xgboost_2/'
sub3 = 'Hyperparameter Search on collection 19 _iraps/'
sub4 = 'Hyperparameter Search on collection 19 _xgbregressor'


In [5]:
def concate_best_result(folder, file_name, scorer, classifier, results):
    path = os.path.join(folder, file_name)
    res = pd.read_csv(path, sep='\t')
    
    res_sort = res.sort_values(['mean_test_'+scorer, 'std_test_'+scorer], ascending=[False, True])
    res_best = res_sort[['mean_test_'+scorer, 'std_test_'+scorer,'params']].head(1).reset_index(drop=True)
    res_best.insert(loc=0, column='dataset', value=file_name[:-11])
    res_best.insert(loc=0, column='classifier', value=classifier)
    if results is None:
        results = res_best
    else:
        results = results.append(res_best, ignore_index=True)
    return results

In [6]:
# best AP scores
files1 = os.listdir(work_dir+sub1)
files2 = os.listdir(work_dir+sub2)
files3 = os.listdir(work_dir+sub3)
files4 = os.listdir(work_dir+sub4)
results = None
scorer = 'binarize_average_precision_scorer'
for fl in files1:
    results = concate_best_result(work_dir+sub1, fl, scorer, 'RandomForestClassifier', results)

for fl in files2:
    results = concate_best_result(work_dir+sub2, fl, scorer, 'XGBClassifier', results)
    
for fl in files3:
    results = concate_best_result(work_dir+sub3, fl, scorer, 'IRAPSClassifier', results)

for fl in files4:
    results = concate_best_result(work_dir+sub4, fl, scorer, 'XGBRegressor', results)


results = results.sort_values(['classifier', 'dataset'])
results

,classifier,dataset,mean_test_binarize_average_precision_scorer,std_test_binarize_average_precision_scorer,params
37,IRAPSClassifier,Afatinib_1032.tsv,0.482563,0.113983,{u'estimator__discretize': -0.5}
35,IRAPSClassifier,Afatinib_1377.tsv,0.477003,0.093717,{u'estimator__discretize': -0.5}
39,IRAPSClassifier,Bortezomib_104.tsv,0.545378,0.126205,{u'estimator__discretize': -0.5}
36,IRAPSClassifier,Cabozantinib_249.tsv,0.346222,0.084752,{u'estimator__discretize': -0.5}
43,IRAPSClassifier,Dasatinib_51.tsv,0.454283,0.137500,{u'estimator__discretize': -0.5}
40,IRAPSClassifier,Erlotinib_1.tsv,0.480096,0.279001,{u'estimator__discretize': -0.5}
34,IRAPSClassifier,Idelalisib_238.tsv,0.483549,0.183466,{u'estimator__discretize': -1.0}
50,IRAPSClassifier,Imatinib_34.tsv,0.436858,0.300380,{u'estimator__discretize': -1.0}
47,IRAPSClassifier,Olaparib_1017.tsv,0.313715,0.121181,{u'estimator__discretize': -0.5}
49,IRAPSClassifier,Olaparib_1495.tsv,0.343326,0.087006,{u'estimator__discretize': -0.5}


In [7]:
# best AP scores
files1 = os.listdir(work_dir+sub1)
files2 = os.listdir(work_dir+sub2)
files3 = os.listdir(work_dir+sub3)
files4 = os.listdir(work_dir+sub4)
results_auc = None
scorer = 'binarize_auc_scorer'
for fl in files1:
    results_auc = concate_best_result(work_dir+sub1, fl, scorer, 'RandomForestClassifier', results_auc)

for fl in files2:
    results_auc = concate_best_result(work_dir+sub2, fl, scorer, 'XGBClassifier', results_auc)
    
for fl in files3:
    results_auc = concate_best_result(work_dir+sub3, fl, scorer, 'IRAPSClassifier', results_auc)

for fl in files4:
    results_auc = concate_best_result(work_dir+sub4, fl, scorer, 'XGBRegressor', results_auc)

results_auc = results_auc.sort_values(['classifier', 'dataset'])
results_auc

,classifier,dataset,mean_test_binarize_auc_scorer,std_test_binarize_auc_scorer,params
37,IRAPSClassifier,Afatinib_1032.tsv,0.848746,0.050332,{u'estimator__discretize': -0.5}
35,IRAPSClassifier,Afatinib_1377.tsv,0.829573,0.039148,{u'estimator__discretize': -0.5}
39,IRAPSClassifier,Bortezomib_104.tsv,0.652592,0.118202,{u'estimator__discretize': -0.5}
36,IRAPSClassifier,Cabozantinib_249.tsv,0.674493,0.080020,{u'estimator__discretize': -1.0}
43,IRAPSClassifier,Dasatinib_51.tsv,0.683538,0.100423,{u'estimator__discretize': -0.5}
40,IRAPSClassifier,Erlotinib_1.tsv,0.758062,0.251170,{u'estimator__discretize': -0.5}
34,IRAPSClassifier,Idelalisib_238.tsv,0.725076,0.111936,{u'estimator__discretize': -1.0}
50,IRAPSClassifier,Imatinib_34.tsv,0.799950,0.154628,{u'estimator__discretize': -1.0}
47,IRAPSClassifier,Olaparib_1017.tsv,0.681698,0.112657,{u'estimator__discretize': -1.0}
49,IRAPSClassifier,Olaparib_1495.tsv,0.595771,0.081338,{u'estimator__discretize': -0.5}


In [8]:
data1 = go.Bar(
    x = results[results['classifier'] == 'IRAPSClassifier']['dataset'],
    y = results[results['classifier'] == 'IRAPSClassifier']['mean_test_binarize_average_precision_scorer'],
    name = 'IRAPS_AP'
)
data2 = go.Bar(
    x = results[results['classifier'] == 'RandomForestClassifier']['dataset'],
    y = results[results['classifier'] == 'RandomForestClassifier']['mean_test_binarize_average_precision_scorer'],
    name = 'RF_AP'
)
data3 = go.Bar(
    x = results[results['classifier'] == 'XGBClassifier']['dataset'],
    y = results[results['classifier'] == 'XGBClassifier']['mean_test_binarize_average_precision_scorer'],
    name = 'XGBC_AP'
)

data4 = go.Bar(
    x = results[results['classifier'] == 'XGBRegressor']['dataset'],
    y = results[results['classifier'] == 'XGBRegressor']['mean_test_binarize_average_precision_scorer'],
    name = 'XGBRegr_AP'
)

data5 = go.Bar(
    x = results_auc[results_auc['classifier'] == 'IRAPSClassifier']['dataset'],
    y = results_auc[results_auc['classifier'] == 'IRAPSClassifier']['mean_test_binarize_auc_scorer'],
    name = 'IRAPS_ROC-AUC'
)
data6 = go.Bar(
    x = results_auc[results_auc['classifier'] == 'RandomForestClassifier']['dataset'],
    y = results_auc[results_auc['classifier'] == 'RandomForestClassifier']['mean_test_binarize_auc_scorer'],
    name = 'RF_ROC-AUC'
)

data7 = go.Bar(
    x = results_auc[results_auc['classifier'] == 'XGBClassifier']['dataset'],
    y = results_auc[results_auc['classifier'] == 'XGBClassifier']['mean_test_binarize_auc_scorer'],
    name = 'XGBC_ROC-AUC'
)
data8 = go.Bar(
    x = results_auc[results_auc['classifier'] == 'XGBRegressor']['dataset'],
    y = results_auc[results_auc['classifier'] == 'XGBRegressor']['mean_test_binarize_auc_scorer'],
    name = 'XGBRegr_ROC-AUC'
)

layout = go.Layout(
    xaxis=dict(
        title='Dataset'
    ),
    yaxis=dict(
        title='Performance score'
    ),
    barmode = 'group'
)
fig = go.Figure(data=[data1, data2, data3, data4], layout=layout)
iplot(fig)

fig = go.Figure(data=[data5, data6, data7,data8], layout=layout)
iplot(fig)
# To show plot, paste the link to this GitHub notebook into http://nbviewer.jupyter.org/

In [9]:
trace1 = {
    "type": 'violin',
    "x": results['classifier'],
    "y": results['mean_test_binarize_average_precision_scorer'],
    "legendgroup": 'AP',
    "scalegroup": 'AP',
    "name": 'AP',
    "box": {
        "visible": True
    },
    "meanline": {
        "visible": True
    },
    "line": {
        "color": 'blue'
    }
}

trace2 = {
    "type": 'violin',
    "x": results_auc['classifier'],
    "y": results_auc['mean_test_binarize_auc_scorer'],
    "legendgroup": 'ROC-AUC',
    "scalegroup": 'ROC-AUC',
    "name": 'ROC-AUC',
    "box": {
        "visible": True
    },
    "meanline": {
        "visible": True
    },
    "line": {
        "color": 'pink'
    }
}

layout = {
    "yaxis": {
        "zeroline": False,
    },
    "violinmode": 'group'
}
fig = go.Figure(data=[trace1, trace2], layout=layout)
iplot(fig)
# To show plot, paste the link to this GitHub notebook into http://nbviewer.jupyter.org/